In [1]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# Get images
# Change to '/data/images/Train/' to use all the 10k images
X = []
for filename in os.listdir('./feature_images/'):
    X.append(img_to_array(load_img('./feature_images/'+filename)))
X = np.array(X, dtype=float)

# Set up train and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

In [3]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='Adam', loss='mse')

In [4]:
# Image transformer
datagen = ImageDataGenerator()

# Generate training data
batch_size = 30
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# Train model      
tensorboard = TensorBoard(log_dir="./output/beta_run1")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=100, steps_per_epoch=2)

Epoch 1/100
2/2 [==============================] - 32s 16s/step - loss: 0.5175
Epoch 2/100
2/2 [==============================] - 30s 15s/step - loss: 0.3793
Epoch 3/100
2/2 [==============================] - 29s 15s/step - loss: 0.0045
Epoch 4/100
2/2 [==============================] - 29s 14s/step - loss: 0.0036
Epoch 5/100
2/2 [==============================] - 29s 14s/step - loss: 0.0033
Epoch 6/100
2/2 [==============================] - 29s 14s/step - loss: 0.0032
Epoch 7/100
2/2 [==============================] - 29s 14s/step - loss: 0.0030
Epoch 8/100
2/2 [==============================] - 30s 15s/step - loss: 0.0031
Epoch 9/100
2/2 [==============================] - 31s 15s/step - loss: 0.0031
Epoch 10/100
2/2 [==============================] - 31s 15s/step - loss: 0.0029
Epoch 11/100
2/2 [==============================] - 31s 16s/step - loss: 0.0027
Epoch 12/100
2/2 [==============================] - 30s 15s/step - loss: 0.0028
Epoch 13/100
2/2 [==============================]

In [5]:
# Test images
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

4/4 [==============================] - 0s 122ms/step
0.000526185787749


In [7]:
color_me = []
for filename in os.listdir('./feature_images/'):
    color_me.append(img_to_array(load_img('./feature_images/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("./result/img1_"+str(i)+".png", lab2rgb(cur))

/home/chanani/anaconda3/lib/python2.7/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [8]:
model.save('model1.h5')

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('model.h5')

In [9]:
import cv2 as cv

In [13]:
cv.imshow('img',cv.imread('./Downloads/I9.jpg'))

error: OpenCV(3.4.1) /opt/conda/conda-bld/opencv-suite_1527005194613/work/modules/highgui/src/window.cpp:636: error: (-2) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Carbon support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function cvShowImage
